In [19]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [20]:
iris = fetch_openml(name='iris', version=1, parser='auto', as_frame=True)
Data = iris.data
target = iris.target

In [21]:
X = Data[:100]
y = target[:100]
label_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
y = y.map(label_mapping)
y = pd.to_numeric(y, errors='coerce').astype(int)
X = X.astype(np.float32)
y = y.astype(np.float32)

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [22]:
@tf.function
def predict(X, W, b):
    return tf.nn.sigmoid(tf.matmul(X, W) + b)

In [23]:
@tf.function
def L(y, y_hat):
    return -(tf.reduce_mean(y*tf.math.log(y_hat) + (1 - y)*tf.math.log(1 - y_hat)))

In [30]:
X_train = tf.constant(X_train)
y_train = tf.constant(y_train)
X_test = tf.constant(X_test)
y_test = tf.constant(y_test)
W = tf.Variable(tf.random.normal(shape=(4, 1), mean=0.0, stddev=1.0, dtype=tf.float32)) 
b = tf.Variable(tf.random.normal(shape=(1,), mean=0.0, stddev=0.001, dtype=tf.float32)) 
alpha = 0.005
for it in range(350):
    with tf.GradientTape() as t:
        current_loss = L(y_train, predict(X_train, W, b))
    print("it", it, current_loss)
    dW, db = t.gradient(current_loss, [W, b])
    W.assign_sub(alpha * dW)
    b.assign_sub(alpha * db)


it 0 tf.Tensor(3.8669488, shape=(), dtype=float32)
it 1 tf.Tensor(3.8113434, shape=(), dtype=float32)
it 2 tf.Tensor(3.755769, shape=(), dtype=float32)
it 3 tf.Tensor(3.700199, shape=(), dtype=float32)
it 4 tf.Tensor(3.6446533, shape=(), dtype=float32)
it 5 tf.Tensor(3.5891235, shape=(), dtype=float32)
it 6 tf.Tensor(3.5336158, shape=(), dtype=float32)
it 7 tf.Tensor(3.4781425, shape=(), dtype=float32)
it 8 tf.Tensor(3.4226935, shape=(), dtype=float32)
it 9 tf.Tensor(3.3672743, shape=(), dtype=float32)
it 10 tf.Tensor(3.311889, shape=(), dtype=float32)
it 11 tf.Tensor(3.2565393, shape=(), dtype=float32)
it 12 tf.Tensor(3.2012446, shape=(), dtype=float32)
it 13 tf.Tensor(3.1459906, shape=(), dtype=float32)
it 14 tf.Tensor(3.0907867, shape=(), dtype=float32)
it 15 tf.Tensor(3.035652, shape=(), dtype=float32)
it 16 tf.Tensor(2.9805763, shape=(), dtype=float32)
it 17 tf.Tensor(2.9255738, shape=(), dtype=float32)
it 18 tf.Tensor(2.8706543, shape=(), dtype=float32)
it 19 tf.Tensor(2.815824, 

In [31]:
y_hat = predict(X_test, W, b)
predicted_labels = (y_hat > 0.5).numpy().astype(int)

df_results = pd.DataFrame({
    'Predicted Probability': np.array(y_hat.numpy()).flatten(),
    'Predicted Label': np.array(predicted_labels).flatten()
})

print(df_results)
mse = tf.reduce_mean(tf.square(y_hat - y_test))
tf.print(mse)


    Predicted Probability  Predicted Label
0                0.536889                1
1                0.603564                1
2                0.352896                0
3                0.523525                1
4                0.368363                0
5                0.504588                1
6                0.366901                0
7                0.623728                1
8                0.484867                0
9                0.479455                0
10               0.491680                0
11               0.525378                1
12               0.618725                1
13               0.383160                0
14               0.560689                1
15               0.438215                0
16               0.687419                1
17               0.562017                1
18               0.570565                1
19               0.509547                1
0.26018706
